<a href="https://colab.research.google.com/github/Ratikant-rout/NLP_CSE-D_ROLLNO-46-ROLLNO-41/blob/main/NLP_CLASS_2_PROJECT_WORDS_EXTRACTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install pymupdf
!pip install nltk

In [11]:
import os
import fitz  # PyMuPDF for extracting text from PDFs
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

# Download required NLTK data
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
# Function to extract text from a PDF
def extract_text_from_pdf(pdf_file):
    doc = fitz.open(pdf_file)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Path to the folder containing your PDF files
pdf_folder_path = '/content/IEEE_FOLDER'  # Adjust this to your folder path

# List of PDF files in the folder
pdf_files = [f for f in os.listdir(pdf_folder_path) if f.endswith('.pdf')]

# Initialize a list to store the document texts
documents = []

# Loop through each PDF file and extract its text
for pdf_file in pdf_files:
    document_path = os.path.join(pdf_folder_path, pdf_file)
    document_text = extract_text_from_pdf(document_path)
    documents.append(document_text)

# Check how many documents are loaded
print(f"Loaded {len(documents)} documents.")



Loaded 3 documents.


In [13]:
# Preprocessing function
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Remove non-alphabetic characters
    text = re.sub(r'[^A-Za-z\s]', '', text)
    text = text.lower()  # Convert text to lowercase
    words = word_tokenize(text)  # Tokenize the text
    words = [word for word in words if word not in stop_words]  # Remove stopwords
    return words

# Preprocess all documents
cleaned_documents = [preprocess_text(doc) for doc in documents]

# Show a preview of the cleaned text (first 20 words of the first document)
print("Cleaned Document Preview:")
print(cleaned_documents[0][:20])  # Preview the first 20 words of the first document



Cleaned Document Preview:
['proceedings', 'machine', 'learning', 'research', 'n', 'machine', 'learning', 'healthcare', 'upstage', 'unsupervised', 'context', 'augmentation', 'utterance', 'classication', 'patientprovider', 'communication', 'june', 'min', 'veronica', 'perezrosas']


In [14]:
# Preprocessing function
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Remove non-alphabetic characters
    text = re.sub(r'[^A-Za-z\s]', '', text)
    text = text.lower()  # Convert text to lowercase
    words = word_tokenize(text)  # Tokenize the text
    words = [word for word in words if word not in stop_words]  # Remove stopwords
    return words

# Preprocess all documents
cleaned_documents = [preprocess_text(doc) for doc in documents]

# Show a preview of the cleaned text (first 20 words of the first document)
print("Cleaned Document Preview:")
print(cleaned_documents[0][:20])  # Preview the first 20 words of the first document



Cleaned Document Preview:
['proceedings', 'machine', 'learning', 'research', 'n', 'machine', 'learning', 'healthcare', 'upstage', 'unsupervised', 'context', 'augmentation', 'utterance', 'classication', 'patientprovider', 'communication', 'june', 'min', 'veronica', 'perezrosas']


In [15]:
# Convert the cleaned documents back to string format for TF-IDF
documents_str = [' '.join(doc) for doc in cleaned_documents]

# Apply TF-IDF to extract the top keywords from all documents
vectorizer = TfidfVectorizer(max_features=10)  # Get top 10 keywords
X = vectorizer.fit_transform(documents_str)

# Get the top keywords
keywords = vectorizer.get_feature_names_out()

# Display the top keywords across all documents
print("Top 10 keywords across all documents:")
for keyword in keywords:
    print(keyword)


Top 10 keywords across all documents:
clinical
concepts
context
contextual
data
ipv
medical
model
models
patients


In [16]:
# Allow the user to input a keyword
user_keyword = input("Enter a keyword to search across the documents: ").lower()

# Check if the entered keyword is in the top keywords
if user_keyword in keywords:
    print(f"The keyword '{user_keyword}' is found in the documents.")
else:
    print(f"The keyword '{user_keyword}' is not found in the top keywords.")

# Calculate relevance scores for each document
keyword_index = keywords.tolist().index(user_keyword) if user_keyword in keywords else None
if keyword_index is not None:
    relevance_scores = X[:, keyword_index].toarray().flatten()

    # Pair the relevance scores with document indices
    doc_scores = list(enumerate(relevance_scores))

    # Sort documents by relevance score in descending order
    sorted_docs = sorted(doc_scores, key=lambda x: x[1], reverse=True)

    # Show the top 5 documents relevant to the keyword
    print("Top 5 documents relevant to the keyword:")
    for idx, score in sorted_docs[:5]:
        print(f"Document {idx + 1}: Relevance score = {score:.4f}")
        print(f"Text: {documents[idx][:500]}...")  # Show a preview of the document
        print('-' * 80)


Enter a keyword to search across the documents: models
The keyword 'models' is found in the documents.
Top 5 documents relevant to the keyword:
Document 1: Relevance score = 0.3925
Text: Proceedings of Machine Learning Research N:1–17, 2020
Machine Learning for Healthcare
UPSTAGE: Unsupervised Context Augmentation for
Utterance Classiﬁcation in Patient-Provider Communication
Do June Min1, Veronica Perez-Rosas1, Shihchen Kuo2, William H. Herman2,
Rada Mihalcea1
dojmin@umich.edu, vrncapr@umich.edu, shihchk@med.umich.edu, wherman@med.umich.edu,
mihalcea@umich.edu
1Department of Electrical Engineering and Computer Science, University of Michigan, Ann Arbor,
MI, USA
2Department of In...
--------------------------------------------------------------------------------
Document 2: Relevance score = 0.1240
Text: Proceedings of Machine Learning Research 106:1–25, 2020
Machine Learning for Healthcare
Fast, Structured Clinical Documentation via Contextual
Autocomplete
Divya Gopinath
divyagop@mit.e

In [17]:
# Allow the user to select a document to see the full text
try:
    doc_choice = int(input("Enter the document number to see the full text (1-3): ")) - 1

    # Check if the document number is valid
    if 0 <= doc_choice < len(documents):
        print(f"\nFull text of Document {doc_choice + 1}:\n")
        # Display a preview of the document (first 500 characters)
        print(documents[doc_choice][:500])  # Show first 500 characters to avoid printing too much
    else:
        print("Invalid document number. Please enter a number between 1 and 3.")
except ValueError:
    print("Invalid input. Please enter a valid integer.")


Enter the document number to see the full text (1-3): 3

Full text of Document 3:

Intimate Partner Violence and Injury Prediction From Radiology Reports
Irene Y. Chen1∗, Emily Alsentzer2, Hyesun Park3, Richard Thomas4, Babina Gosangi5,
Rahul Gujrathi3, and Bharti Khurana3,6
1Electrical Engineering and Computer Science, Massachusetts Institute of Technology,
Cambridge, MA 02139, USA
2Health Sciences and Technology, Massachusetts Institute of Technology,
Cambridge, MA 02139, USA
3Department of Radiology, Brigham and Women’s Hospital, Boston, MA 02115, USA
4Department of Radiolo
